# گام اول

### آماده‌سازی کولب

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# !rm -r spark*
!ls

gdrive	sample_data


In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [5]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
sc = spark.sparkContext
spark

## بخش اول

پس از راه ‌اندازی کولب جهت کار با اسپارک، داده‌ها را از طریق گوگل درایو لود می‌کنیم. سپس به کمک توابعی که در کد قابل مشاهده است، جملات را به کلمات تکی جدا می‌کنیم. و در مرحله‌ی بعدی، هر کلکه‌ای در فایل مشاهده می‌کنیم را به دوتایی (کلمه،۱) تبدیل می‌کنیم. در نهایت با شمارش اسن دوتایی های مشابه، تعداد  تکرار هر کلمه را در متن می‌یابیم. قسمتی از خروجی در نتیجه کد سلول بعدی قابل مشاهده است. از آنجایی که نمایش کامل خروجی در اینجا میسر نبود، نتیجه را یک فایل تکست ذخیره کرده‌ام و ضمیمه شده است

In [ ]:
def punctuation_eliminator(word):
  import re 
  x = re.sub("[^a-zA-Z]+", "", word)
  return x

# loading data
text = spark.sparkContext.textFile('/content/gdrive/My Drive/bigData_hw3/q1/input.txt')

# removing punctuation by using punctuation_eliminator fn.
words = text.flatMap(lambda line: line.split(" ")).map(punctuation_eliminator)

# mapping (word, 1): each word give us a (word, 1) 
maped_words = words.map(lambda word: (word, 1))
print('Number of words in input: ', maped_words.count())
print('---'*15)

### counting distinct word
DistinctWordsCount = maped_words.reduceByKey(lambda a,b:a +b)
print('Number of distinct words in input: ',DistinctWordsCount.count())
print('---'*15)

### counting repetition of each word
SortedWordsCount = DistinctWordsCount.map(lambda a: (a[1], a[0])).sortByKey("desc")
print('sorted words:', SortedWordsCount.top(50))

# saving ouput in a textfile as a RDD file but we do NOT use it... because we want all results in one text file. so we use
# SortedWordsCount.saveAsTextFile('/content/gdrive/My Drive/bigData_hw3/q1/result_game1_part1.txt')
# saving output as a list in a text file in colab
SortedWordsCount_list = DistinctWordsCount.map(lambda a: (a[1], a[0])).sortByKey(ascending=False).collect()
with open('/content/gdrive/My Drive/bigData_hw3/q1/result_game1_part1.txt', 'w') as writefile:
  for element in SortedWordsCount_list:
    writefile.write(str(element) + '\n')

print('---'*15)
print('Result has been saved in google drive in a text file named "result_game1_part1.txt"')

Number of words in input:  5043
---------------------------------------------
Number of distinct words in input:  1308
---------------------------------------------
sorted words: [(312, 'the'), (237, 'and'), (162, 'of'), (152, 'to'), (116, 'in'), (106, 'a'), (76, 'is'), (60, 'that'), (57, 'people'), (52, 'they'), (52, 'it'), (44, 'this'), (44, ''), (41, 'was'), (41, 'I'), (40, 'on'), (39, 'have'), (38, 'with'), (33, 'you'), (33, 'them'), (32, 'are'), (30, 'This'), (29, 'were'), (28, 'all'), (27, 'for'), (25, 'be'), (24, 'up'), (24, 'can'), (24, 'as'), (23, 'time'), (23, 'The'), (21, 'when'), (21, 'what'), (20, 'game'), (20, 'about'), (19, 'which'), (19, 'their'), (19, 'like'), (19, 'from'), (19, 'but'), (18, 'will'), (18, 'who'), (18, 'more'), (18, 'get'), (18, 'because'), (17, 'way'), (17, 'one'), (17, 'has'), (17, 'London'), (16, 'so')]
Result has been saved in google drive in a text file named "result_game1_part1.txt"


## بخش دوم

باری یافتن تعداد کلماتی که با ام انگلیسی شروع می‌شوند از کدی که در سلول بعدی آن را آورده ام استفاده می‌کنیم. بعد از اعمال فیلتر روی حرف اول کلمات، مشاببه قسمت قبل شروع به شمار تعداد تکرار کلمات می‌کنیم. در نهایت خروجی را در یک فایل تکت ذخیره می‌کنیم. خروجی شامل کللماتی که با حرف ام شروع می‌شپند و تعداد تکرار آن‌ها است



we need words that start with "m" or "M". so, using below code in order to filter on first letter of each word :

> words.filter(lambda x: x.startswith('m') or x.startswith('M'))





In [ ]:
# A function in order to remove punctuation
def punctuation_eliminator(word):
  import re 
  x = re.sub("[^a-zA-Z]+", "", word)
  return x

# loading data
text = spark.sparkContext.textFile('/content/gdrive/My Drive/bigData_hw3/q1/input.txt')

### counting all words
# removing punctuation by using punctuation_eliminator fn.
words = text.flatMap(lambda line: line.split(" ")).map(punctuation_eliminator)

# getting the words that start with M or m:
words_starts_with_m = words.filter(lambda x: x.startswith('m') or x.startswith('M'))
print("Number of words that starst with 'm' or 'M': ", words_starts_with_m.count())
print('\n', '---'*15, '\n')
print("some words that starts with 'm' or 'M' :", '\n')
print(words_starts_with_m.take(100))

# saving the output in a text file in colab
with open('/content/gdrive/My Drive/bigData_hw3/q1/result_game1_part2.txt', 'w') as writefile:
  for element in words_starts_with_m.collect():
    writefile.write(str(element) + '\n')

print('Result has been saved in google drive in a text file named "result_game1_part2.txt"')

Number of words that starst with 'm' or 'M':  150

 --------------------------------------------- 

some words that starts with 'm' or 'M' : 

['make', 'make', 'mastery', 'many', 'me', 'make', 'me', 'me', 'my', 'more', 'minute', 'might', 'me', 'me', 'my', 'moves', 'me', 'my', 'my', 'my', 'make', 'my', 'me', 'me', 'more', 'more', 'members', 'more', 'made', 'much', 'more', 'me', 'make', 'me', 'more', 'me', 'mastering', 'merely', 'method', 'my', 'me', 'money', 'Marjane', 'many', 'many', 'more', 'magical', 'may', 'more', 'material', 'majority', 'Marjane', 'Marjane', 'many', 'more', 'Marjane', 'more', 'meaning', 'monochromatic', 'much', 'more', 'more', 'meant', 'martyred', 'made', 'more', 'Marjane', 'my', 'much', 'more', 'most', 'me', 'Many', 'majority', 'much', 'make', 'much', 'making', 'many', 'Magog', 'myths', 'most', 'major', 'means', 'moving', 'modern', 'most', 'my', 'most', 'many', 'Many', 'made', 'most', 'move', 'many', 'moved', 'moved', 'modern', 'me', 'most']
Result has been saved 

## بخش سوم

ابتدا علايم نگارسی را حذف می‌کنیم، سپس به کمک فیلتر کلماتی که دارای صول ۵ هستند را به مرحله‌ی بعد می‌فرستیم (منظور از کلمات با طول ۵، کلمات ۶ حرفی است). سپس به کمک فیلتر کلماتی که با حروف صدا دار شروع می‌شوند را حذف می‌کنیم. درنهایت با انجام عمل ردیوس تعداد کلمات را می‌شماریم. بخشی از خروجی در نتیجه سلول زیر قابل مشاهده است؛ خروجی کامل در فایل متنی ضمیمه شده است

In [ ]:
import re 
def punctuation_eliminator(word):  # A fn in order to remove punctuation marks.
  x = re.sub("[^a-zA-Z]+", "", word)
  return x

def not_vowel_initializer(input_word): # A fn in order to remove words that start with vowel sounds.
  if input_word[0] not in 'aeiouAEIOU':
    return input_word 

# loading data
text = spark.sparkContext.textFile('/content/gdrive/My Drive/bigData_hw3/q1/input.txt')

# removing punctuation by using punctuation_eliminator fn.
words = text.flatMap(lambda line: line.split(" ")).map(punctuation_eliminator)

# getting the words with five letter
five_letter_words = words.filter(lambda x: len(x) == 5)

# removing word that start with vowel sounds
not_vowel_initializer_words_five_letter = five_letter_words.filter(not_vowel_initializer)

# map-reduce job
map_reduced = not_vowel_initializer_words_five_letter.map(lambda word: (word, 1)).reduceByKey(lambda a,b:a +b).map(lambda a: (a[1], a[0])).sortByKey(ascending=False)
map_reduced.take(30)

# printing
print("Number of word that contains 5 letters and does not contain a vowel word in first letter: ", not_vowel_initializer_words_five_letter.count())
print('---'*15, '\n')
print("some words that contains 5 letters and does not contain a vowel word in first letter:", '\n')
print(map_reduced.take(30), '\n')

# saving the output in a text file in colab
with open('/content/gdrive/My Drive/bigData_hw3/q1/result_game1_part3_tuple.txt', 'w') as writefile:
  for element in map_reduced.collect():
    writefile.write(str(element) + '\n')
print('---'*20, '\n' 'Result has been saved in google drive in a text file named "result_game1_part3_tuple.txt"')

Number of word that contains 5 letters and does not contain a vowel word in first letter:  487
--------------------------------------------- 

some words that contains 5 letters and does not contain a vowel word in first letter: 

[(19, 'which'), (19, 'their'), (15, 'games'), (15, 'could'), (14, 'these'), (13, 'style'), (12, 'where'), (12, 'think'), (11, 'there'), (11, 'place'), (10, 'would'), (9, 'great'), (7, 'class'), (7, 'There'), (7, 'being'), (6, 'Buddy'), (6, 'These'), (6, 'lying'), (6, 'times'), (5, 'three'), (5, 'world'), (5, 'first'), (5, 'while'), (5, 'hence'), (5, 'spend'), (4, 'story'), (4, 'point'), (4, 'quite'), (4, 'based'), (4, 'young')] 

------------------------------------------------------------ 
Result has been saved in google drive in a text file named "result_game1_part3_tuple.txt"


## بخش چهارم

### a) using part one to extract stop words

ابتدا به کمک بخش اول کلماتی که جزء ۱۰٪ کلمات پرتکرار هستندد را می‌یابیم و آن‌ها را به عنوان ایست واژه‌ها در نظر می‌گیریم. برای این منظور دوتایی‌های کلمات و تعداد تکرار آن‌ها را متناسب با تعداد تکرا سورت می‌کنیم و سپس ۱۰٪ ابتدایی لیست سورت شده را به عنوان ایست واژه در نظر می‌گیریم

In [ ]:
def punctuation_eliminator(word):
  import re 
  x = re.sub("[^a-zA-Z]+", "", word)
  return x

# loading data
text = spark.sparkContext.textFile('/content/gdrive/My Drive/bigData_hw3/q1/input.txt')

### counting all words
# removing punctuation by using punctuation_eliminator fn.
words = text.flatMap(lambda line: line.split(" ")).map(punctuation_eliminator)

# mapping (word, 1): each word give us a (word, 1) 
maped_words = words.map(lambda word: (word, 1))

### counting distinct word
DistinctWordsCount = maped_words.reduceByKey(lambda a,b:a +b)

### counting repetition of each word
SortedWordsCount = DistinctWordsCount.map(lambda a: (a[1], a[0])).sortByKey("desc")
number_of_stopWords = int(SortedWordsCount.count() * 0.1)

# finding stop words
stopWords_tuple = SortedWordsCount.top(number_of_stopWords)
stopWords = [ a[1] for a in stopWords_tuple ]

### b) 

سپس فایل ورودی را می‌گیریم. در هر خط ایست واه‌ها و علايم نگارشی را حذف می‌کنیم. در نهایت یک متن همانند متن ورودی را به عنوان خروجی بازمی‌گردانیم. البته در خروجی ایست واه‌ها و علايم نگارشی حذف شده‌اند. توجه داریم که در نتیجه‌ی سلول بعدی مقادیر داخل لیت قرار گرفته اند که صرفا برای تست درستی روش می‌باشد. در نهایت خروجی در فایل متنی ذخیره شده است. فایل نیز ضمیمه شده است

In [ ]:
import re
def removing_stop_words_and_punctuation(text):  
  '''
  this function take a sentence as input.
  at first, remove all non alphabet characters.
  then, remove all word that are stop word (we use stop words from global variable).
  finally, edited sentence is returned
  '''
  global stopWords

  # removing non-alphabet characters
  removed_non_alphabet = re.sub("[^a-zA-Z]+", " ", text)

  # putting each word of sentece in list. each word is a list element
  list_removed_non_alphabet = removed_non_alphabet.split()

  # making a new blank string in order to add words into it. finally, this string will be edited sentence.
  edited_text = ''
  for word in list_removed_non_alphabet:
    if word not in stopWords:
      edited_text += word + ' '
  return edited_text

# loading data
text = spark.sparkContext.textFile('/content/gdrive/My Drive/bigData_hw3/q1/input.txt')

# take each sentence and edit it (editing refer removing punctuation and stopWords)
sentences = text.flatMap(lambda line: line.split(".")).map(removing_stop_words_and_punctuation)

# testing the method by comparison of input and edited sentece
print('testing the method by comparison of some input and edited sentece:')
print('input text:')
print(text.flatMap(lambda line: line.split(".")).take(2))   ## input
print('ouput text:')
print(sentences.take(2), '\n')                          ## edited

# saving the output in a text file in colab;
with open('/content/gdrive/My Drive/bigData_hw3/q1/result_game1_part4.txt', 'w') as writefile:
  for element in sentences.collect() :
    writefile.write(str(element) )
print('---'*20, '\n', 'Result has been saved in google drive in a text file named "result_game1_part4.txt"')

testing the method by comparison of some input and edited sentece:
input text:
['Games are a fun way to get people involved and learning in a happy environment and get them to work on concepts and tactics without them knowing it a lot of the time', ' Because of this, these games were perfect in a class on negotiation and persuasion because it loosened people up and allowed them to learn in a fun environment']
ouput text:
['Games fun involved learning happy environment work concepts tactics without knowing lot ', 'Because perfect negotiation persuasion loosened allowed learn fun environment '] 

------------------------------------------------------------ 
 Result has been saved in google drive in a text file named "result_game1_part4.txt"


## بخش پنجم

جهت پیدا کردن دو کلمه‌ای هایی که پشت هم قرار گرفته اند، از کد زیر استفاده می‌کنیم

> flatMap(lambda xs: (tuple(x) for x in zip(xs, xs[1:])))

سپس همانند مرحله‌های قبلی این دوتایی ها را می‌شماریم. قسمتی از خروجی در نتیجه سلول بعدی نمایش داده شده است و خروجی کامل در فایل متنی ضمیمه شده است

In [ ]:
import re 
def punctuation_eliminator(word):  # A fn in order to remove punctuation marks.
  x = re.sub("[^a-zA-Z]+", " ", word)
  return x

# loading data
text = spark.sparkContext.textFile('/content/gdrive/My Drive/bigData_hw3/q1/input.txt')

# removing punctuation by using punctuation_eliminator fn.
text_no_punctuation = text.map(punctuation_eliminator)

# putting bigrams and rep in a tuple
bigrams = text_no_punctuation.map(lambda line: line.strip().split(" ")) \
                   .flatMap(lambda xs: (tuple(x) for x in zip(xs, xs[1:])))

# sorting by value; filtering if value is bigger than one.
sorted_bigrams = bigrams.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y).sortBy(lambda a: a[1], ascending=False).filter(lambda a: a[1]>1)

# printing a small sample of result
print('some sorted bigrams for sample:', '\n')
print(count_bigrams.take(100), '\n')

# saving the output in a text file in colab
with open('/content/gdrive/My Drive/bigData_hw3/q1/result_game1_part5.txt', 'w') as writefile:
  for element in sorted_bigrams.collect():
    writefile.write(str(element) + '\n')
print('---'*20,'\n','Result has been saved in google drive in a text file named "result_game1_part5.txt"')


some sorted bigrams for sample: 

[(('of', 'the'), 56), (('in', 'the'), 48), (('and', 'the'), 15), (('on', 'the'), 12), (('all', 'of'), 12), (('to', 'the'), 11), (('Soho', 'Square'), 11), (('of', 'this'), 10), (('people', 'to'), 10), (('of', 'a'), 10), (('to', 'get'), 9), (('with', 'the'), 9), (('in', 'a'), 9), (('it', 'is'), 9), (('them', 'to'), 8), (('This', 'is'), 8), (('the', 'square'), 8), (('that', 'I'), 7), (('the', 'game'), 7), (('this', 'is'), 7), (('for', 'the'), 7), (('is', 'the'), 7), (('one', 'of'), 7), (('the', 'way'), 7), (('have', 'been'), 7), (('in', 'London'), 7), (('in', 'this'), 6), (('will', 'be'), 6), (('that', 'they'), 6), (('and', 'then'), 6), (('everyone', 'and'), 6), (('have', 'a'), 6), (('is', 'a'), 6), (('it', 'was'), 6), (('as', 'a'), 6), (('gender', 'and'), 6), (('the', 'Romans'), 6), (('get', 'people'), 5), (('the', 'time'), 5), (('of', 'them'), 5), (('Fuck', 'Your'), 5), (('I', 'm'), 5), (('people', 'and'), 5), (('want', 'to'), 5), (('with', 'people'), 5